In [15]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
import numpy as np


In [16]:
#!pip install psycopg2

In [17]:
path ='usa_county_wise.csv'
path2 ='unemployment.csv'
data_df=pd.read_csv(path)
data_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",1/22/2020,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",1/22/2020,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",1/22/2020,0,0
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,"Puerto Rico, US",1/22/2020,0,0
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,"Virgin Islands, US",1/22/2020,0,0


In [18]:
state_infection =data_df[["Province_State","Country_Region","Date","Confirmed"]]
#state_infection.dtypes

#df['Date']= pd.to_datetime(df['Date']) 

In [19]:
state_infection["month"] = state_infection["Date"].str[0:2].str.replace("/","").copy()
state_infection.sample() 

C:\Users\momad\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Province_State,Country_Region,Date,Confirmed,month
63213,Mississippi,US,2/10/2020,0,2


In [32]:
infectionByMonth= state_infection.loc[(state_infection["month"] == "1")|(state_infection["month"] == "2"), :] 
infectionByMonth=state_infection.loc[(state_infection["Country_Region"] == "US"), :] 

infectionByMonth.head()

,Province_State,Country_Region,Date,Confirmed,month
0,American Samoa,US,1/22/2020,0,1
1,Guam,US,1/22/2020,0,1
2,Northern Mariana Islands,US,1/22/2020,0,1
3,Puerto Rico,US,1/22/2020,0,1
4,Virgin Islands,US,1/22/2020,0,1


In [33]:
infectionByMonth

,Province_State,Country_Region,Date,Confirmed,month
0,American Samoa,US,1/22/2020,0,1
1,Guam,US,1/22/2020,0,1
2,Northern Mariana Islands,US,1/22/2020,0,1
3,Puerto Rico,US,1/22/2020,0,1
4,Virgin Islands,US,1/22/2020,0,1
...,...,...,...,...,...
256982,Washington,US,4/9/2020,838,4
256983,West Virginia,US,4/9/2020,0,4
256984,Wisconsin,US,4/9/2020,0,4
256985,Wyoming,US,4/9/2020,0,4


In [34]:
infectionByMonth = infectionByMonth.rename(columns={"Province_State": "region", "Date": "time_period", "Confirmed": "confirmed", "Country_Region": "country_region", "month": "month_no"})
infectionByMonth


,region,country_region,time_period,confirmed,month_no
0,American Samoa,US,1/22/2020,0,1
1,Guam,US,1/22/2020,0,1
2,Northern Mariana Islands,US,1/22/2020,0,1
3,Puerto Rico,US,1/22/2020,0,1
4,Virgin Islands,US,1/22/2020,0,1
...,...,...,...,...,...
256982,Washington,US,4/9/2020,838,4
256983,West Virginia,US,4/9/2020,0,4
256984,Wisconsin,US,4/9/2020,0,4
256985,Wyoming,US,4/9/2020,0,4


In [43]:
#infection['ID'] = np.arange(infection.shape[0])
#infection_df = infection.head()
#infection_df

In [72]:
unemployment = pd.read_csv(path2)
unemployment.head()


,State,Jan.,Feb.
0,Alabama,2.7,2.7
1,Alaska,6.0,5.8
2,Arizona,4.5,4.5
3,Arkansas,3.5,3.9
4,California,3.9,2.5


In [70]:
#unemployment_df['ID'] = np.arange(unemployment_df.shape[0])
#unemployment_df.head()

In [73]:
unemployment_df = unemployment_df.rename(columns={"State": "region","Jan.": "january", "Feb.": "february"})
unemployment_df.head()

,region,january,february
0,Alabama,2.7,2.7
1,Alaska,6.0,5.8
2,Arizona,4.5,4.5
3,Arkansas,3.5,3.9
4,California,3.9,2.5


### Connectto local database

In [74]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_project_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Chech for tables

In [75]:
engine.table_names()

['infection', 'unemployment']

In [76]:
['infection, unemployment']

['infection, unemployment']

### Use pandas to load csv converted DataFrame into database

In [60]:
infectionByMonth.to_sql(name='infection', con=engine, if_exists='append', index=False)

In [77]:
unemployment_df.to_sql(name='unemployment', con=engine, if_exists='append', index=False)